In [ ]:
#INSTALACIÓN DE LIBRERIAS
!pip install Torch==1.8.1
!pip install TorchVision
!pip install pickle
#IMPORTS
import os
import shutil
import cv2
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler
import pickle

In [ ]:
import pickle
import random

ruta_train = 'C:\\Users\\Angel\\Downloads\\IDC_train.dat'
ruta_test = 'C:\\Users\\Angel\\Downloads\\IDC_test.dat'

# Cargar el archivo de entrenamiento
with open(ruta_train, 'rb') as f:
    (X_train, y_train) = pickle.load(f)

# Cargar el archivo de prueba
with open(ruta_test, 'rb') as f:
    (X_test, y_test) = pickle.load(f)

# Obtener el 30% de los datos de entrenamiento
total_samples_train = len(X_train)
num_samples_train = int(0.3 * total_samples_train)
random_indices_train = random.sample(range(total_samples_train), num_samples_train)
X_train = [X_train[i] for i in random_indices_train]
y_train = [y_train[i] for i in random_indices_train]

# Obtener el 30% de los datos de prueba
total_samples_test = len(X_test)
num_samples_test = int(0.3 * total_samples_test)
random_indices_test = random.sample(range(total_samples_test), num_samples_test)
X_test = [X_test[i] for i in random_indices_test]
y_test = [y_test[i] for i in random_indices_test]

# Realizar cualquier otra operación necesaria con los datos cargados


In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create train and test datasets
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

# Define the model architecture
class Model(nn.Module):
    def __init__(self, num_neurons):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, num_neurons, 3)
        self.conv2 = nn.Conv2d(num_neurons, num_neurons*2, 3)
        self.conv3 = nn.Conv2d(num_neurons*2, num_neurons*4, 3)
        self.conv4 = nn.Conv2d(num_neurons*4, num_neurons*8, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(num_neurons*8, num_neurons)  # Asume que después de cuatro convoluciones y max pooling, la dimensión es num_neurons * 8 * 3 * 3
        self.fc2 = nn.Linear(num_neurons, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Define the function to create the model
def crear_modelo(num_neurons):
    print("CREAR MODELO", num_neurons)
    model = Model(num_neurons)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters())
    return model, criterion, optimizer

# Define the checkpoint directory
checkpoint_dir = 'D:/Master/TFM/Checkpoints'

# Create checkpoint directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

def save_ckpt(model, optimizer, epoch, num_neurons, checkpoint_dir):
    save_dir = os.path.join(checkpoint_dir, "checkpoint_{:04d}_{}.pth".format(epoch, num_neurons))

    cp = {
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'epoch': epoch,
    }

    torch.save(cp, save_dir)
    print("Saved checkpoint into {}".format(save_dir))


def load_ckpt(checkpoint_dir, num_neurons):
    last_epoch = 0
    checkpoint_file = ''

    for file in os.listdir(checkpoint_dir):
        if file.endswith('{}.pth'.format(num_neurons)):
            epoch_number = int(file.split('_')[1])
            if epoch_number > last_epoch:
                last_epoch = epoch_number
                checkpoint_file = file

    if last_epoch == 0:
        return None, 0
    else:
        print("Loaded checkpoint", os.path.join(checkpoint_dir, checkpoint_file))
        checkpoint = torch.load(os.path.join(checkpoint_dir, checkpoint_file))
        return checkpoint, checkpoint['epoch']

# Define the grid of parameters
param_grid = {
    'num_neurons': [10, 20, 30],
}

# Perform grid search
best_accuracy = 0.0
best_params = None

for params in ParameterGrid(param_grid):
    print("Using {} neurons".format(params['num_neurons']))
    model, criterion, optimizer = crear_modelo(params['num_neurons'])
    checkpoint, start_epoch = load_ckpt(checkpoint_dir, params['num_neurons'])
    if checkpoint is not None:
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])

    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

    for epoch in range(start_epoch, start_epoch + 10):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.permute((0,3,1,2))
            outputs = model(inputs)
            loss = criterion(outputs, labels.float().unsqueeze(1))
            loss.backward()
            optimizer.step()

        save_ckpt(model, optimizer, epoch + 1, params['num_neurons'], checkpoint_dir) # Save the checkpoint

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.permute((0,3,1,2))
                outputs = model(inputs)
                predicted = (outputs >= 0.5).squeeze().long()
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print("Epoch: {}, Accuracy: {}".format(epoch + 1, accuracy))
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

print("Best Accuracy:", best_accuracy)
print("Best Parameters:", best_params)
